In [1]:
cd set2

/media/datastorage/Phong/ufc101/set2


In [2]:
ls -l

total 21966880
-rw-rw-r-- 1 bribeiro bribeiro        113 jan 23 15:20 checkpoint
-rw-rw-r-- 1 bribeiro bribeiro  138864569 jan 22 00:50 DenseNet201_CNNRNN2-set2.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro        836 jan 22 00:50 DenseNet201_CNNRNN2-set2.index
-rw-rw-r-- 1 bribeiro bribeiro 2868085648 jan 19 21:06 DenseNet201_CNNRNN_test_100_set2.pickle
-rw-rw-r-- 1 bribeiro bribeiro      30000 jan 19 21:08 DenseNet201_CNNRNN_test_labels_set2.npy
-rw-rw-r-- 1 bribeiro bribeiro 7363006852 jan 19 21:05 DenseNet201_CNNRNN_train_100_set2.pickle
-rw-rw-r-- 1 bribeiro bribeiro      76816 jan 19 21:08 DenseNet201_CNNRNN_train_labels_set2.npy
-rw-rw-r-- 1 bribeiro bribeiro 1066511519 jan 23 15:20 DenseNet201_Transformer-set2.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro       4586 jan 23 15:20 DenseNet201_Transformer-set2.index
-rw-rw-r-- 1 bribeiro bribeiro  143058873 jan 21 23:56 ResNet152_CNNRNN2-set2.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro        836 jan 21 23:56 ResN

In [14]:
# %mv DenseNet201_CNNRNN_train_labels.npy DenseNet201_CNNRNN_train_labels_set2.npy
# %mv DenseNet201_CNNRNN_test_labels.npy DenseNet201_CNNRNN_test_labels_set2.npy
# %mv Dense201_CNNRNN_test_100_set2.pickle DenseNet201_CNNRNN_test_100_set2.pickle
# %mv Dense201_CNNRNN_train_100_set2.pickle DenseNet201_CNNRNN_train_100_set2.pickle

In [3]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [4]:
IMG_SIZE = 224
# BATCH_SIZE = 32
EPOCHS = 20

MAX_SEQ_LENGTH = 100
NUM_FEATURES = 1920

In [5]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 9586
Total videos for testing: 3734


,video_name,tag
3747,v_HeadMassage_g20_c02.avi,HeadMassage
6142,v_PlayingPiano_g15_c04.avi,PlayingPiano
6199,v_PlayingSitar_g03_c02.avi,PlayingSitar
7432,v_ShavingBeard_g18_c04.avi,ShavingBeard
8257,v_SumoWrestling_g18_c02.avi,SumoWrestling
2775,v_FieldHockeyPenalty_g16_c01.avi,FieldHockeyPenalty
4464,v_JumpingJack_g01_c07.avi,JumpingJack
8207,v_StillRings_g25_c06.avi,StillRings
2315,v_CricketShot_g20_c07.avi,CricketShot
9298,v_WallPushups_g04_c02.avi,WallPushups


In [5]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)


In [6]:

def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet201(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input 

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

2023-01-21 23:11:00.739834: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-21 23:11:01.270167: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


In [6]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BlowingCandles', 'BodyWeightSquats', 'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'BreastStroke', 'BrushingTeeth', 'CleanAndJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen', 'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 'FrisbeeCatch', 'FrontCrawl', 'GolfSwing', 'Haircut', 'HammerThrow', 'Hammering', 'HandstandPushups', 'HandstandWalking', 'HeadMassage', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow', 'JugglingBalls', 'JumpRope', 'JumpingJack', 'Kayaking', 'Knitting', 'LongJump', 'Lunges', 'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars', 'PizzaTossing', 'PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'P

2023-01-22 22:53:57.507188: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-22 22:53:58.422967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


In [12]:
# from tqdm import tqdm

# def prepare_all_videos(df, root_dir):
#     num_samples = len(df)
#     video_paths = df["video_name"].values.tolist()
#     labels = df["tag"].values
#     labels = label_processor(labels[..., None]).numpy()

#     # `frame_masks` and `frame_features` are what we will feed to our sequence model.
#     # `frame_masks` will contain a bunch of booleans denoting if a timestep is
#     # masked with padding or not.
#     frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
#     frame_features = np.zeros(
#         shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
#     )

#     # For each video.
#     for idx, path in tqdm(enumerate(video_paths)):
#         # Gather all its frames and add a batch dimension.
#         frames = load_video(os.path.join(root_dir, path))
#         frames = frames[None, ...]

#         # Initialize placeholders to store the masks and features of the current video.
#         temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
#         temp_frame_features = np.zeros(
#             shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
#         )

#         # Extract features from the frames of the current video.
#         for i, batch in enumerate(frames):
#             video_length = batch.shape[0]
#             length = min(MAX_SEQ_LENGTH, video_length)
#             for j in range(length):
#                 temp_frame_features[i, j, :] = feature_extractor.predict(
#                     batch[None, j, :]
#                 )
#             temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

#         frame_features[idx,] = temp_frame_features.squeeze()
#         frame_masks[idx,] = temp_frame_mask.squeeze()

#     return (frame_features, frame_masks), labels


# # train_data, train_labels = prepare_all_videos(train_df, "train")
# test_data, test_labels = prepare_all_videos(test_df, "test")

# # print(f"Frame features in train set: {train_data[0].shape}")
# # print(f"Frame masks in train set: {train_data[1].shape}")

0it [00:00, ?it/s]2022-07-10 00:05:40.497486: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2022-07-10 00:05:41.463905: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-10 00:05:41.464543: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-10 00:05:41.464602: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-07-10 00:05:41.465321: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-10 00:05:41.465430: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
3783it [7:12:23,  6.86s/it]


In [14]:
# train_data, train_labels = prepare_all_videos(train_df, "train")

9537it [18:34:16,  7.01s/it]


In [ ]:
# # Utility for our sequence model.
# # Bidirectional

# def get_sequence_model():
#     class_vocab = label_processor.get_vocabulary()

#     frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#     mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

#     # Refer to the following tutorial to understand the significance of using `mask`:
#     # https://keras.io/api/layers/recurrent_layers/gru/
#     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5))(#16, 512
#         frame_features_input, mask=mask_input
#     )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
#     x = keras.layers.Dense(1024)(x)#8, 256
#     x = keras.layers.LeakyReLU()(x)
#     output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

#     rnn_model = keras.Model([frame_features_input, mask_input], output)

#     rnn_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
#     )
#     return rnn_model


# # Utility for running experiments.
# def run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     seq_model = get_sequence_model()
#     print(seq_model.summary())
    
#     history = seq_model.fit(
#         [train_data[0], train_data[1]],
#         train_labels,
#         validation_split=0.2,
#         epochs=EPOCHS,
#         callbacks=[checkpoint],
#     )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# _, sequence_model = run_experiment()

In [ ]:
#40.13% LSTM(2048,dropout=0.1)
#46.47% LSTM(2048,dropout=0.5)
#43.96% LSTM(4096,dropout=0.5)

In [ ]:
# class_vocab = label_processor.get_vocabulary()

# frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
# mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

# # Refer to the following tutorial to understand the significance of using `mask`:
# # https://keras.io/api/layers/recurrent_layers/gru/
# x = keras.layers.LSTM(2048, return_sequences=True,dropout=0.1)(#16, 512
#     frame_features_input, mask=mask_input
# )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
# x = keras.layers.Dense(1024, activation="relu")(x)#8, 256
# output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

# rnn_model = keras.Model([frame_features_input, mask_input], output)

# rnn_model.compile(
#     loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# )

# print(rnn_model.summary())

In [ ]:
# class_vocab = label_processor.get_vocabulary()

# frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
# mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

# # Refer to the following tutorial to understand the significance of using `mask`:
# # https://keras.io/api/layers/recurrent_layers/gru/
# x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.1)(#16, 512
#     frame_features_input, mask=mask_input
# )
# # x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
# x = keras.layers.Dense(1024, activation="relu")(x)#8, 256
# output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

# rnn_model = keras.Model([frame_features_input, mask_input], output)

# rnn_model.compile(
#     loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# )

# print(rnn_model.summary())

In [ ]:

# def prepare_single_video(frames):
#     frames = frames[None, ...]
#     frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
#     frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

#     for i, batch in enumerate(frames):
#         video_length = batch.shape[0]
#         length = min(MAX_SEQ_LENGTH, video_length)
#         for j in range(length):
#             frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
#         frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

#     return frame_features, frame_mask


# def sequence_prediction(path):
#     class_vocab = label_processor.get_vocabulary()

#     frames = load_video(os.path.join("test", path))
#     frame_features, frame_mask = prepare_single_video(frames)
#     probabilities = sequence_model.predict([frame_features, frame_mask])[0]

#     for i in np.argsort(probabilities)[::-1]:
#         print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
#     return frames


# # This utility is for visualization.
# # Referenced from:
# # https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
# def to_gif(images):
#     converted_images = images.astype(np.uint8)
#     imageio.mimsave("animation.gif", converted_images, fps=10)
#     return embed.embed_file("animation.gif")


# test_video = np.random.choice(test_df["video_name"].values.tolist())
# print(f"Test video path: {test_video}")
# test_frames = sequence_prediction(test_video)
# to_gif(test_frames[:MAX_SEQ_LENGTH])

In [ ]:
ls -l

In [ ]:
# import numpy as np
# import tensorflow as tf 

# np.save('Inception_CNNRNN_train_data.npy', train_data)    # .npy extension is added if not given
# # np.save('Inception_CNNRNN_test_data.npy', test_data[0])    # .npy extension is added if not given
# # np.save('Inception_CNNRNN_train_labels.npy', train_data[1])    # .npy extension is added if not given
# # np.save('Inception_CNNRNN_test_labels.npy', test_labels)    # .npy extension is added if not given

In [ ]:
# print(train_data[0].shape)
# print(train_data[1].shape)

In [ ]:
# print(train_data.type)

In [15]:
# import pickle

# with open('Dense201_CNNRNN_train_100.pickle', 'wb') as f:
#     pickle.dump(train_data, f)

In [13]:
# import pickle

# with open('Dense201_CNNRNN_test_100.pickle', 'wb') as f:
#     pickle.dump(test_data, f)

In [ ]:
# import numpy as np
# import tensorflow as tf 

# # np.save('densenet_train_data.npy', train_data)    # .npy extension is added if not given
# # np.save('densenet_test_data.npy', test_data)    # .npy extension is added if not given
# np.save('CNNRNN_train_labels.npy', train_labels)    # .npy extension is added if not given
# np.save('CNNRNN_test_labels.npy', test_labels)    # .npy extension is added if not given

In [ ]:
# ls -l

In [ ]:
# mv Inception_CNNRNN_20.pickle Inception_CNNRNN_train_20.pickle

In [13]:
ls -l

total 20925352
-rw-rw-r-- 1 bribeiro bribeiro        101 jan 21 00:00 checkpoint
-rw-rw-r-- 1 bribeiro bribeiro  138864596 jan 20 21:36 DenseNet201_CNNRNN2-set2.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro        836 jan 20 21:36 DenseNet201_CNNRNN2-set2.index
-rw-rw-r-- 1 bribeiro bribeiro 2868085648 jan 19 21:06 DenseNet201_CNNRNN_test_100_set2.pickle
-rw-rw-r-- 1 bribeiro bribeiro      30000 jan 19 21:08 DenseNet201_CNNRNN_test_labels.npy
-rw-rw-r-- 1 bribeiro bribeiro 7363006852 jan 19 21:05 DenseNet201_CNNRNN_train_100_set2.pickle
-rw-rw-r-- 1 bribeiro bribeiro      76816 jan 19 21:08 DenseNet201_CNNRNN_train_labels.npy
-rw-rw-r-- 1 bribeiro bribeiro  143058900 jan 21 00:00 ResNet152_CNNRNN2-set2.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro        836 jan 21 00:00 ResNet152_CNNRNN2-set2.index
-rw-rw-r-- 1 bribeiro bribeiro 3059266448 jan 20 00:49 Resnet152_CNNRNN_test_100_set2.pickle
-rw-rw-r-- 1 bribeiro bribeiro      30000 jan 20 00:55 Resnet152_CNNRNN_test_labels_se

In [7]:
import numpy as np
import tensorflow as tf 
import pickle

with open('DenseNet201_CNNRNN_train_100_set2.pickle', 'rb') as f:
     d_train_data = pickle.load(f)
with open('DenseNet201_CNNRNN_test_100_set2.pickle', 'rb') as f:
     d_test_data = pickle.load(f)
        
# d_train_data = np.load('densenet_train_data.npy')
# d_test_data = np.load('densenet_test_data.npy')
d_train_labels = np.load('DenseNet201_CNNRNN_train_labels_set2.npy')
d_test_labels = np.load('DenseNet201_CNNRNN_test_labels_set2.npy')

In [ ]:
# label_processor = keras.layers.StringLookup(
#     num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
# )
# # print(label_processor.get_vocabulary())

# def get_sequence_model():
#     class_vocab = label_processor.get_vocabulary()

#     frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#     mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

#     # Refer to the following tutorial to understand the significance of using `mask`:
#     # https://keras.io/api/layers/recurrent_layers/gru/
#     x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
#         frame_features_input, mask=mask_input
#     )    
# #     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
# #         frame_features_input, mask=mask_input
# #     )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
#     x = keras.layers.Dense(1024)(x)#8, 256
#     x = keras.layers.GaussianNoise(0.4)(x)
#     x = keras.layers.LeakyReLU(0.1)(x)    
#     output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

#     rnn_model = keras.Model([frame_features_input, mask_input], output)

# #     rnn_model.compile(
# #         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# #     )
#     return rnn_model

# # Utility for running experiments.
# def d_run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     seq_model = get_sequence_model()
#     print(seq_model.summary())
    
#     optimizer = keras.optimizers.SGD(lr=1e-3)
# #     optimizer = keras.optimizers.Adadelta()
    
#     seq_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
#     )    
    
#     history = seq_model.fit(
#         [d_train_data[0], d_train_data[1]],
#         d_train_labels,
# #         validation_split=0.2,
#         validation_data=([d_test_data[0], d_test_data[1]],
#         d_test_labels),
#         epochs=100,
# #         epochs=EPOCHS,
#         callbacks=[checkpoint],
#     )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([d_test_data[0], d_test_data[1]], d_test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# d_run_experiment()

In [ ]:
#lr=0.1 78.24%
#lr=0.01 79.20%
#lr=0.001 77.98

In [ ]:
# label_processor = keras.layers.StringLookup(
#     num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
# )
# # print(label_processor.get_vocabulary())

# def get_sequence_model():
#     class_vocab = label_processor.get_vocabulary()

#     frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#     mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

#     # Refer to the following tutorial to understand the significance of using `mask`:
#     # https://keras.io/api/layers/recurrent_layers/gru/
#     x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
#         frame_features_input, mask=mask_input
#     )    
# #     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
# #         frame_features_input, mask=mask_input
# #     )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
#     x = keras.layers.Dense(1024)(x)#8, 256
#     x = keras.layers.GaussianNoise(0.4)(x)
#     x = keras.layers.LeakyReLU(0.1)(x)    
#     output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

#     rnn_model = keras.Model([frame_features_input, mask_input], output)

# #     rnn_model.compile(
# #         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# #     )
#     return rnn_model

# # Utility for running experiments.
# def d_run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     seq_model = get_sequence_model()
#     print(seq_model.summary())
    

#     lr_schedule = keras.callbacks.LearningRateScheduler(
#                   lambda epoch: 1e-6 * 10**(4*epoch / 10))
    
#     optimizer = keras.optimizers.SGD(lr=1e-3)
    
#     seq_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
#     )    
# #     seq_model.compile(optimizer=optimizer,
# #                       loss='categorical_crossentropy',
# #                      metrics=['accuracy'])     
    
#     history = seq_model.fit(
#         [d_train_data[0], d_train_data[1]],
#         d_train_labels,
# #         validation_split=0.2,
#         validation_data=([d_test_data[0], d_test_data[1]],
#         d_test_labels),
#         epochs=20,
# #         epochs=EPOCHS,
#         callbacks=[lr_schedule,checkpoint],
#     )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([d_test_data[0], d_test_data[1]], d_test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# his, seq = d_run_experiment()


In [ ]:
from matplotlib import pyplot as plt

plt.semilogx(his.history['lr'], his.history['accuracy'])
plt.axis([1e-6, 1, 0, 1])
plt.xlabel('lr')
plt.ylabel('accuracy')
plt.show()

In [ ]:
print(d_train_data[0].shape)
print(d_train_data[1].shape)

In [ ]:
print(d_train_data[1][0][39])

In [ ]:
# import numpy as np

# # set length of video sequence and indices
# MAX_SEQ_LENGTH = 20#override max sequence length
# #max_seq=40
# # copy_indices=[0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38]
# # copy_indices=[0,4,8,12,16,20,24,28,32,36]
# # copy_indices=[0,5,10,15,20,25,30,35]
# # copy_indices=[0,8,16,24,32]
# # copy_indices=[0,19]
# # copy_indices=[0]
# # max_seq=100
# # # copy_indices=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,
# #               26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,
# #               51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,
# #               76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99]
# copy_indices=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95]
# # copy_indices=[0,10,20,30,40,50,60,70,80,90]
# # copy_indices=[0,20,40,60,80]
# # copy_indices=[0,25,50,75]
# # copy_indices=[0,50]
# d_train_data2_1 = np.copy(d_train_data[0][:,copy_indices, ])
# d_train_data2_2 = np.copy(d_train_data[1][:,copy_indices, ])
# d_train_data2 = (d_train_data2_1,d_train_data2_2)

# d_test_data2_1 = np.copy(d_test_data[0][:,copy_indices, ])
# d_test_data2_2 = np.copy(d_test_data[1][:,copy_indices, ])
# d_test_data2 = (d_test_data2_1,d_test_data2_2)
# # print(d_train_data2.shape)

# label_processor = keras.layers.StringLookup(
#     num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
# )
# # print(label_processor.get_vocabulary())

# def get_sequence_model():
#     class_vocab = label_processor.get_vocabulary()

#     frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#     mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

#     # Refer to the following tutorial to understand the significance of using `mask`:
#     # https://keras.io/api/layers/recurrent_layers/gru/
#     x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
#         frame_features_input, mask=mask_input
#     )    
# #     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
# #         frame_features_input, mask=mask_input
# #     )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
#     x = keras.layers.Dense(1024)(x)#8, 256
#     x = keras.layers.GaussianNoise(0.4)(x)
#     x = keras.layers.LeakyReLU(0.1)(x)    
#     output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

#     rnn_model = keras.Model([frame_features_input, mask_input], output)

# #     rnn_model.compile(
# #         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# #     )
#     return rnn_model

# # Utility for running experiments.
# def d_run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     seq_model = get_sequence_model()
#     print(seq_model.summary())
    
#     optimizer = keras.optimizers.SGD(lr=1e-2)
# #     optimizer = keras.optimizers.Adadelta()
    
#     seq_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
#     )    
    
#     history = seq_model.fit(
#         [d_train_data2[0], d_train_data2[1]],
#         d_train_labels,
# #         validation_split=0.2,
#         validation_data=([d_test_data2[0], d_test_data2[1]],
#         d_test_labels),
#         epochs=100,
# #         epochs=EPOCHS,
#         callbacks=[checkpoint],
#     )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([d_test_data2[0], d_test_data2[1]], d_test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# d_run_experiment()

2022-07-13 23:47:20.257446: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-13 23:47:20.733553: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 1920)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 2048)         32514048    ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 1024)         2098176     ['lstm[0][0]']               

/home/bribeiro/miniconda3/envs/ar/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
2022-07-13 23:47:28.427893: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1464883200 exceeds 10% of free system memory.
2022-07-13 23:47:29.369089: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1464883200 exceeds 10% of free system memory.


Epoch 1/100


2022-07-13 23:47:32.893093: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


299/299 [==============================] - ETA: 0s - loss: 3.5424 - accuracy: 0.2839

2022-07-13 23:47:43.656385: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 581068800 exceeds 10% of free system memory.
2022-07-13 23:47:44.043870: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 581068800 exceeds 10% of free system memory.



Epoch 1: val_loss improved from inf to 2.09063, saving model to /tmp/video_classifier
299/299 [==============================] - 17s 46ms/step - loss: 3.5424 - accuracy: 0.2839 - val_loss: 2.0906 - val_accuracy: 0.4811
Epoch 2/100
299/299 [==============================] - ETA: 0s - loss: 1.1348 - accuracy: 0.7590
Epoch 2: val_loss did not improve from 2.09063
299/299 [==============================] - 12s 39ms/step - loss: 1.1348 - accuracy: 0.7590 - val_loss: 9.7352 - val_accuracy: 0.0650
Epoch 3/100
299/299 [==============================] - ETA: 0s - loss: 0.5579 - accuracy: 0.8842
Epoch 3: val_loss did not improve from 2.09063
299/299 [==============================] - 12s 39ms/step - loss: 0.5579 - accuracy: 0.8842 - val_loss: 6.1651 - val_accuracy: 0.1758
Epoch 4/100
299/299 [==============================] - ETA: 0s - loss: 0.3040 - accuracy: 0.9382
Epoch 4: val_loss improved from 2.09063 to 0.78710, saving model to /tmp/video_classifier
299/299 [==============================

In [ ]:
#max seq = 40
#seq 1 75.20%
#seq 2 79.04%
#seq 5 80.33%
#seq 8 80.44%
#seq 10 79.75%
#seq 20 79.78%

#max seq = 100
#seq 2 79.91
#seq 4 81.52
#seq 5 81.95
#seq 10 81.52
#seq 20 81.13

In [6]:
import numpy as np
import tensorflow as tf 
import pickle

with open('DenseNet201_CNNRNN_train_100_set2.pickle', 'rb') as f:
     d_train_data = pickle.load(f)
with open('DenseNet201_CNNRNN_test_100_set2.pickle', 'rb') as f:
     d_test_data = pickle.load(f)
        
# d_train_data = np.load('densenet_train_data.npy')
# d_test_data = np.load('densenet_test_data.npy')
d_train_labels = np.load('DenseNet201_CNNRNN_train_labels_set2.npy')
d_test_labels = np.load('DenseNet201_CNNRNN_test_labels_set2.npy')

In [7]:
train_labels = d_train_labels
test_labels = d_test_labels

In [37]:
import numpy as np

MAX_SEQ_LENGTH = 10#override max sequence length
#max_seq=40
# copy_indices=[0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38]
copy_indices=[0,4,8,12,16,20,24,28,32,36]
# copy_indices=[0,5,10,15,20,25,30,35]
# copy_indices=[0,8,16,24,32]
# copy_indices=[0,19]
# copy_indices=[0]
# max_seq=100
# # copy_indices=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,
#               26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,
#               51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,
#               76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99]
# copy_indices=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95]
# copy_indices=[0,10,20,30,40,50,60,70,80,90]
#copy_indices=[0,20,40,60,80]
# copy_indices=[0,25,50,75]
# copy_indices=[0,50]
train_data = np.copy(d_train_data[0][:,copy_indices, ])
test_data = np.copy(d_test_data[0][:,copy_indices, ])
print(train_data.shape)

(9586, 10, 1920)


In [9]:

class PositionalEmbedding(keras.layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = keras.layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask


In [10]:

class TransformerEncoder(keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [keras.layers.Dense(dense_dim, activation=tf.nn.gelu), keras.layers.Dense(embed_dim),]
        )
        self.layernorm_1 = keras.layers.LayerNormalization()
        self.layernorm_2 = keras.layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


In [42]:
# Label preprocessing with StringLookup.
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"]), mask_token=None
)

def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 64
    num_heads = 6
    classes = len(label_processor.get_vocabulary())

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = keras.layers.GlobalMaxPooling1D()(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


def run_experiment():
    filepath = "DenseNet201_Transformer-set2"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    model = get_compiled_model()
    history = model.fit(
        train_data,
        train_labels,
        # validation_split=0.15,
        validation_data=([test_data, test_labels]),
        epochs=100,
        callbacks=[checkpoint],
    )
    
    print(f"Test best accuracy: {round(max(history.history['val_accuracy'])*100,2)}%")
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     model.load_weights(filepath)
#     _, accuracy = model.evaluate(test_data, test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model

trained_model = run_experiment()

Epoch 1/100
299/300 [============================>.] - ETA: 0s - loss: 5.6159 - accuracy: 0.0407
Epoch 1: val_loss improved from inf to 3.02976, saving model to DenseNet201_Transformer-set2
300/300 [==============================] - 23s 74ms/step - loss: 5.6120 - accuracy: 0.0411 - val_loss: 3.0298 - val_accuracy: 0.2544
Epoch 2/100
299/300 [============================>.] - ETA: 0s - loss: 1.7728 - accuracy: 0.5495
Epoch 2: val_loss improved from 3.02976 to 1.53537, saving model to DenseNet201_Transformer-set2
300/300 [==============================] - 20s 68ms/step - loss: 1.7706 - accuracy: 0.5502 - val_loss: 1.5354 - val_accuracy: 0.6360
Epoch 3/100
299/300 [============================>.] - ETA: 0s - loss: 0.7366 - accuracy: 0.7939
Epoch 3: val_loss did not improve from 1.53537
300/300 [==============================] - 14s 48ms/step - loss: 0.7367 - accuracy: 0.7940 - val_loss: 1.5606 - val_accuracy: 0.6762
Epoch 4/100
299/300 [============================>.] - ETA: 0s - loss: 0.

Epoch 30/100
299/300 [============================>.] - ETA: 0s - loss: 0.0535 - accuracy: 0.9834
Epoch 30: val_loss did not improve from 1.29990
300/300 [==============================] - 15s 51ms/step - loss: 0.0534 - accuracy: 0.9834 - val_loss: 2.1202 - val_accuracy: 0.7327
Epoch 31/100
299/300 [============================>.] - ETA: 0s - loss: 0.0499 - accuracy: 0.9851
Epoch 31: val_loss did not improve from 1.29990
300/300 [==============================] - 15s 51ms/step - loss: 0.0499 - accuracy: 0.9851 - val_loss: 2.1633 - val_accuracy: 0.7209
Epoch 32/100
299/300 [============================>.] - ETA: 0s - loss: 0.0458 - accuracy: 0.9858
Epoch 32: val_loss did not improve from 1.29990
300/300 [==============================] - 15s 51ms/step - loss: 0.0458 - accuracy: 0.9858 - val_loss: 2.4137 - val_accuracy: 0.7065
Epoch 33/100
299/300 [============================>.] - ETA: 0s - loss: 0.0383 - accuracy: 0.9889
Epoch 33: val_loss did not improve from 1.29990
300/300 [========

300/300 [==============================] - 15s 50ms/step - loss: 0.0091 - accuracy: 0.9970 - val_loss: 2.3105 - val_accuracy: 0.7263
Epoch 60/100
299/300 [============================>.] - ETA: 0s - loss: 0.0233 - accuracy: 0.9944
Epoch 60: val_loss did not improve from 1.29990
300/300 [==============================] - 15s 51ms/step - loss: 0.0232 - accuracy: 0.9944 - val_loss: 1.8790 - val_accuracy: 0.7263
Epoch 61/100
299/300 [============================>.] - ETA: 0s - loss: 0.0158 - accuracy: 0.9960
Epoch 61: val_loss did not improve from 1.29990
300/300 [==============================] - 15s 50ms/step - loss: 0.0158 - accuracy: 0.9960 - val_loss: 2.2229 - val_accuracy: 0.7413
Epoch 62/100
299/300 [============================>.] - ETA: 0s - loss: 0.0284 - accuracy: 0.9916
Epoch 62: val_loss did not improve from 1.29990
300/300 [==============================] - 15s 51ms/step - loss: 0.0284 - accuracy: 0.9917 - val_loss: 1.8831 - val_accuracy: 0.7386
Epoch 63/100
299/300 [========

Epoch 89/100
299/300 [============================>.] - ETA: 0s - loss: 0.0116 - accuracy: 0.9967
Epoch 89: val_loss did not improve from 1.29990
300/300 [==============================] - 15s 51ms/step - loss: 0.0116 - accuracy: 0.9967 - val_loss: 2.7936 - val_accuracy: 0.7442
Epoch 90/100
299/300 [============================>.] - ETA: 0s - loss: 0.0194 - accuracy: 0.9941
Epoch 90: val_loss did not improve from 1.29990
300/300 [==============================] - 15s 51ms/step - loss: 0.0193 - accuracy: 0.9942 - val_loss: 2.7903 - val_accuracy: 0.7322
Epoch 91/100
299/300 [============================>.] - ETA: 0s - loss: 0.0089 - accuracy: 0.9964
Epoch 91: val_loss did not improve from 1.29990
300/300 [==============================] - 15s 51ms/step - loss: 0.0089 - accuracy: 0.9965 - val_loss: 2.4495 - val_accuracy: 0.7357
Epoch 92/100
299/300 [============================>.] - ETA: 0s - loss: 0.0316 - accuracy: 0.9907
Epoch 92: val_loss did not improve from 1.29990
300/300 [========

In [ ]:
# 40 frames
#max_seq = 2
#head=4
#dense_dim=32 74.53
#64 = 74.26
#128 = 74.29
#1024 = 70.73

#head=6
#16 = 74.48
#32 = 73.43
#64 = 74.37
#128 = 74.21

#max_seq = 10
#head=6
#32 = 74.64 
#64 = 75.55
#128 = 76.0

# 100 frames
#max_seq = 4
#head=1
#dense_dim=4 69.47
#8 = 69.63
#16 = 68.53
#32 = 71.0
#64 = 69.52
#128 = 69.42

##max_seq = 2
#head=1
#dense_dim=4 69.42
#128 = 69.68

#head=4
#128 = 75.15
#64 = 75.39

#head=8
#128 = 74.75
#256 = 74.75

##max_seq = 10
#head=4
#dim=128 = 75.33
#64 = 76.00

#head=6
#64 = 77.16
#32 = 76.73
#1024 = 73.57
#16 = 76.62
#8 = 76.57

##max_seq =20
#head=6
#8 = 77.34
#32 = 75.79
#64 = 76.0
#128 = 75.87
#256 = 76.43

In [8]:

# class TransformerEncoder2(keras.layers.Layer):
#     def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
#         super().__init__(**kwargs)
#         self.embed_dim = embed_dim
#         self.dense_dim = dense_dim
#         self.num_heads = num_heads
#         self.attention = keras.layers.MultiHeadAttention(
#             num_heads=num_heads, key_dim=embed_dim, dropout=0.3
#         )
#         self.dense_proj = keras.Sequential(
#             [keras.layers.Dense(dense_dim, activation=tf.nn.gelu), 
#              keras.layers.Dense(embed_dim),]
#         )
#         self.layernorm_1 = keras.layers.LayerNormalization()
#         self.layernorm_2 = keras.layers.LayerNormalization()

#     def call(self, inputs, mask=None):
#         if mask is not None:
#             mask = mask[:, tf.newaxis, :]

#         #layer 1
#         attention_output = self.attention(inputs, inputs, attention_mask=mask)
#         proj_input = self.layernorm_1(inputs + attention_output)
#         proj_output = self.dense_proj(proj_input)
#         layer1_output = self.layernorm_2(proj_input + proj_output)
        
#         #layer 2
#         attention_output2 = self.attention(layer1_output, layer1_output)
#         proj_input2 = self.layernorm_1(layer1_output + attention_output2)
#         proj_output2 = self.dense_proj(proj_input2)
#         layer2_output = self.layernorm_2(proj_input2 + proj_output2)
#         return layer2_output


In [22]:

# class TransformerEncoder3(keras.layers.Layer):
#     def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
#         super().__init__(**kwargs)
#         self.embed_dim = embed_dim
#         self.dense_dim = dense_dim
#         self.num_heads = num_heads
#         ##########
        
# #         self.attention = keras.layers.MultiHeadAttention(
# #             num_heads=num_heads, key_dim=embed_dim, dropout=0.3
# #         )
# #         self.dense_proj = keras.Sequential(
# #             [keras.layers.Dense(dense_dim, activation=tf.nn.gelu), keras.layers.Dense(embed_dim),]
# #         )
# #         self.layernorm_1 = keras.layers.LayerNormalization()
# #         self.layernorm_2 = keras.layers.LayerNormalization()

#     def call(self, inputs, mask=None):
#         if mask is not None:
#             mask = mask[:, tf.newaxis, :]

# #         attention_output = self.attention(inputs, inputs, attention_mask=mask)
# #         proj_input = self.layernorm_1(inputs + attention_output)
# #         proj_output = self.dense_proj(proj_input)
# #         return self.layernorm_2(proj_input + proj_output)

#         attention_output = keras.layers.MultiHeadAttention(
#             num_heads=self.num_heads, key_dim=self.embed_dim, dropout=0.1
#         )(inputs, inputs, attention_mask=mask)

#         proj_input = keras.layers.LayerNormalization()(inputs + attention_output)
#         proj_output = keras.Sequential(
#             [keras.layers.Dense(self.dense_dim, activation=tf.nn.gelu), keras.layers.Dense(self.embed_dim),]
#         ) (proj_input)
#         return keras.layers.LayerNormalization()(proj_input + proj_output)
# #         x1 = keras.layers.LayerNormalization()(inputs)
# #         attention_output = keras.layers.MultiHeadAttention(
# #             num_heads=self.num_heads, key_dim=self.embed_dim // self.num_heads, dropout=0.1
# #         )(x1, x1)

# #         # Skip connection
# #         x2 = keras.layers.Add()([attention_output, inputs])

# #         # Layer Normalization and MLP
# #         x3 = keras.layers.LayerNormalization()(x2)
# #         x3 = keras.Sequential(
# #             [
# #                 keras.layers.Dense(units=self.embed_dim * 4, activation=tf.nn.gelu),
# #                 keras.layers.Dense(units=self.embed_dim, activation=tf.nn.gelu),
# #             ]
# #         )(x3)

# #         # Skip connection
# #         encoded_patches = keras.layers.Add()([x3, x2])

# #         # Layer normalization and Global average pooling.
# #         representation = keras.layers.LayerNormalization()(encoded_patches)
# # #         representation = layers.GlobalAvgPool1D()(representation)   
# #         return representation


In [ ]:
# # Label preprocessing with StringLookup.
# label_processor = keras.layers.StringLookup(
#     num_oov_indices=0, vocabulary=np.unique(train_df["tag"]), mask_token=None
# )

# def get_compiled_model():
#     sequence_length = MAX_SEQ_LENGTH
#     embed_dim = NUM_FEATURES
#     dense_dim = 16
#     num_heads = 6
#     classes = len(label_processor.get_vocabulary())

#     inputs = keras.Input(shape=(None, None))
#     x = PositionalEmbedding(
#         sequence_length, embed_dim, name="frame_position_embedding"
#     )(inputs)
# #     x = TransformerEncoder2(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
# #     x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
# #     x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer2")(x)
# #     x = TransformerEncoder3(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)

#     attention_output = keras.layers.MultiHeadAttention(
#         num_heads=num_heads, key_dim=embed_dim, dropout=0.1
#     )(x, x)

#     proj_input = keras.layers.LayerNormalization()(x + attention_output)
#     proj_output = keras.Sequential(
#         [keras.layers.Dense(dense_dim, activation=tf.nn.gelu), keras.layers.Dense(embed_dim),]
#     ) (proj_input)
#     representation = keras.layers.LayerNormalization()(proj_input + proj_output)

#     representation = keras.layers.GlobalMaxPooling1D()(representation)
#     representation = keras.layers.Dropout(0.5)(representation)
#     outputs = keras.layers.Dense(classes, activation="softmax")(representation)
#     model = keras.Model(inputs, outputs)

#     model.compile(
#         optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
#     )
#     return model


# def run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     model = get_compiled_model()
#     history = model.fit(
#         train_data,
#         train_labels,
#         # validation_split=0.15,
#         validation_data=([test_data, test_labels]),
#         epochs=100,
#         callbacks=[checkpoint],
#     )
    
#     print(f"Test best accuracy: {round(max(history.history['val_accuracy'])*100,2)}%")
# #     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

# #     model.load_weights(filepath)
# #     _, accuracy = model.evaluate(test_data, test_labels)
# #     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return model

# trained_model = run_experiment()

In [14]:
# # Label preprocessing with StringLookup.
# label_processor = keras.layers.StringLookup(
#     num_oov_indices=0, vocabulary=np.unique(train_df["tag"]), mask_token=None
# )

# def get_compiled_model():
#     sequence_length = MAX_SEQ_LENGTH
#     embed_dim = NUM_FEATURES
#     dense_dim = 16
#     num_heads = 6
#     classes = len(label_processor.get_vocabulary())

#     inputs = keras.Input(shape=(None, None))
#     x = PositionalEmbedding(
#         sequence_length, embed_dim, name="frame_position_embedding"
#     )(inputs)
# #     x = TransformerEncoder2(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
# #     x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
# #     x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer2")(x)
# #     x = TransformerEncoder3(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)

# #     attention_output = keras.layers.MultiHeadAttention(
# #         num_heads=num_heads, key_dim=embed_dim, dropout=0.1
# #     )(x, x)

# #     proj_input = keras.layers.LayerNormalization()(x + attention_output)
# #     proj_output = keras.Sequential(
# #         [keras.layers.Dense(dense_dim, activation=tf.nn.gelu), keras.layers.Dense(embed_dim),]
# #     ) (proj_input)
# #     representation = keras.layers.LayerNormalization()(proj_input + proj_output)

#     # Layer normalization and MHSA
#     encoded_patches = x
    
#     for _ in range(2):
# #         x1 = keras.layers.LayerNormalization()(encoded_patches)
#         x1 = encoded_patches
#         attention_output = keras.layers.MultiHeadAttention(
#             num_heads=num_heads, key_dim=embed_dim, dropout=0.1
#         )(x1, x1)

#         # Skip connection
#         x2 = keras.layers.Add()([attention_output, x1])

#         # Layer Normalization and MLP
#         x3 = keras.layers.LayerNormalization(epsilon=1e-6)(x2)
#         x3 = keras.Sequential(
#             [
#                 keras.layers.Dense(units=dense_dim, activation=tf.nn.gelu),
#                 keras.layers.Dense(units=embed_dim),
#             ]
#         )(x3)

#         # Skip connection
#         encoded_patches = keras.layers.Add()([x3, x2])
#         encoded_patches = keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

#     representation = keras.layers.GlobalMaxPooling1D()(encoded_patches)
#     representation = keras.layers.Dropout(0.5)(representation)
    
#     outputs = keras.layers.Dense(classes, activation="softmax")(representation)
#     model = keras.Model(inputs, outputs)

#     model.compile(
#         optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
#     )
#     return model


# def run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     model = get_compiled_model()
#     history = model.fit(
#         train_data,
#         train_labels,
#         # validation_split=0.15,
#         validation_data=([test_data, test_labels]),
#         epochs=200,
#         callbacks=[checkpoint],
#     )
    
#     print(f"Test best accuracy: {round(max(history.history['val_accuracy'])*100,2)}%")
# #     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

# #     model.load_weights(filepath)
# #     _, accuracy = model.evaluate(test_data, test_labels)
# #     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return model

# trained_model = run_experiment()

Epoch 1/200
298/299 [============================>.] - ETA: 0s - loss: 5.9908 - accuracy: 0.0110
Epoch 1: val_loss improved from inf to 4.83479, saving model to /tmp/video_classifier
299/299 [==============================] - 41s 132ms/step - loss: 5.9909 - accuracy: 0.0110 - val_loss: 4.8348 - val_accuracy: 0.0169
Epoch 2/200
298/299 [============================>.] - ETA: 0s - loss: 4.6088 - accuracy: 0.0311
Epoch 2: val_loss improved from 4.83479 to 4.12286, saving model to /tmp/video_classifier
299/299 [==============================] - 34s 112ms/step - loss: 4.6086 - accuracy: 0.0311 - val_loss: 4.1229 - val_accuracy: 0.0375
Epoch 3/200
298/299 [============================>.] - ETA: 0s - loss: 3.8247 - accuracy: 0.0720
Epoch 3: val_loss improved from 4.12286 to 3.53855, saving model to /tmp/video_classifier
299/299 [==============================] - 36s 121ms/step - loss: 3.8248 - accuracy: 0.0720 - val_loss: 3.5385 - val_accuracy: 0.0841
Epoch 4/200
298/299 [====================

298/299 [============================>.] - ETA: 0s - loss: 1.1398 - accuracy: 0.6443
Epoch 28: val_loss did not improve from 1.88266
299/299 [==============================] - 30s 101ms/step - loss: 1.1397 - accuracy: 0.6443 - val_loss: 2.0155 - val_accuracy: 0.5041
Epoch 29/200
298/299 [============================>.] - ETA: 0s - loss: 1.0726 - accuracy: 0.6652
Epoch 29: val_loss improved from 1.88266 to 1.82766, saving model to /tmp/video_classifier
299/299 [==============================] - 35s 116ms/step - loss: 1.0725 - accuracy: 0.6652 - val_loss: 1.8277 - val_accuracy: 0.5152
Epoch 30/200
298/299 [============================>.] - ETA: 0s - loss: 1.1124 - accuracy: 0.6478
Epoch 30: val_loss did not improve from 1.82766
299/299 [==============================] - 30s 100ms/step - loss: 1.1123 - accuracy: 0.6478 - val_loss: 2.0001 - val_accuracy: 0.5075
Epoch 31/200
298/299 [============================>.] - ETA: 0s - loss: 1.0096 - accuracy: 0.6808
Epoch 31: val_loss improved from

298/299 [============================>.] - ETA: 0s - loss: 0.4754 - accuracy: 0.8441
Epoch 57: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.4754 - accuracy: 0.8441 - val_loss: 2.1052 - val_accuracy: 0.5699
Epoch 58/200
298/299 [============================>.] - ETA: 0s - loss: 0.4171 - accuracy: 0.8628
Epoch 58: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.4171 - accuracy: 0.8628 - val_loss: 2.2376 - val_accuracy: 0.5707
Epoch 59/200
298/299 [============================>.] - ETA: 0s - loss: 0.4243 - accuracy: 0.8597
Epoch 59: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.4245 - accuracy: 0.8596 - val_loss: 2.5511 - val_accuracy: 0.5300
Epoch 60/200
298/299 [============================>.] - ETA: 0s - loss: 0.5019 - accuracy: 0.8378
Epoch 60: val_loss did not improve from 1.74361
299/299 [==================

298/299 [============================>.] - ETA: 0s - loss: 0.2149 - accuracy: 0.9306
Epoch 86: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.2149 - accuracy: 0.9306 - val_loss: 2.8169 - val_accuracy: 0.5710
Epoch 87/200
298/299 [============================>.] - ETA: 0s - loss: 0.1889 - accuracy: 0.9344
Epoch 87: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.1891 - accuracy: 0.9343 - val_loss: 3.4052 - val_accuracy: 0.5630
Epoch 88/200
298/299 [============================>.] - ETA: 0s - loss: 0.2976 - accuracy: 0.9117
Epoch 88: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.2976 - accuracy: 0.9117 - val_loss: 3.1263 - val_accuracy: 0.5826
Epoch 89/200
298/299 [============================>.] - ETA: 0s - loss: 0.2307 - accuracy: 0.9251
Epoch 89: val_loss did not improve from 1.74361
299/299 [==================

298/299 [============================>.] - ETA: 0s - loss: 0.1849 - accuracy: 0.9381
Epoch 115: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.1849 - accuracy: 0.9381 - val_loss: 3.3021 - val_accuracy: 0.5921
Epoch 116/200
298/299 [============================>.] - ETA: 0s - loss: 0.1654 - accuracy: 0.9436
Epoch 116: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.1654 - accuracy: 0.9436 - val_loss: 3.0589 - val_accuracy: 0.6045
Epoch 117/200
298/299 [============================>.] - ETA: 0s - loss: 0.1480 - accuracy: 0.9499
Epoch 117: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.1480 - accuracy: 0.9499 - val_loss: 3.3075 - val_accuracy: 0.6011
Epoch 118/200
298/299 [============================>.] - ETA: 0s - loss: 0.1437 - accuracy: 0.9493
Epoch 118: val_loss did not improve from 1.74361
299/299 [===========

298/299 [============================>.] - ETA: 0s - loss: 0.1118 - accuracy: 0.9619
Epoch 144: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.1117 - accuracy: 0.9619 - val_loss: 3.6659 - val_accuracy: 0.6032
Epoch 145/200
298/299 [============================>.] - ETA: 0s - loss: 0.1107 - accuracy: 0.9631
Epoch 145: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.1107 - accuracy: 0.9631 - val_loss: 3.9792 - val_accuracy: 0.6059
Epoch 146/200
298/299 [============================>.] - ETA: 0s - loss: 0.0858 - accuracy: 0.9702
Epoch 146: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.0858 - accuracy: 0.9702 - val_loss: 3.8132 - val_accuracy: 0.5953
Epoch 147/200
298/299 [============================>.] - ETA: 0s - loss: 0.1129 - accuracy: 0.9604
Epoch 147: val_loss did not improve from 1.74361
299/299 [===========

298/299 [============================>.] - ETA: 0s - loss: 0.0787 - accuracy: 0.9747
Epoch 173: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.0787 - accuracy: 0.9747 - val_loss: 5.0990 - val_accuracy: 0.5691
Epoch 174/200
298/299 [============================>.] - ETA: 0s - loss: 0.1147 - accuracy: 0.9653
Epoch 174: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.1146 - accuracy: 0.9653 - val_loss: 4.3982 - val_accuracy: 0.5892
Epoch 175/200
298/299 [============================>.] - ETA: 0s - loss: 0.0776 - accuracy: 0.9734
Epoch 175: val_loss did not improve from 1.74361
299/299 [==============================] - 30s 101ms/step - loss: 0.0775 - accuracy: 0.9734 - val_loss: 5.1513 - val_accuracy: 0.5815
Epoch 176/200
298/299 [============================>.] - ETA: 0s - loss: 0.0759 - accuracy: 0.9761
Epoch 176: val_loss did not improve from 1.74361
299/299 [===========

In [ ]:
#layer=1
#head=6
#dim=16 75.73%
#layer=5 75.05%

#head=1
#layer=2 66.56%

#head=4
#layer=2 63.36%


In [ ]:
#num_layer=2
#max_seq=100
#num_frame=10
#head=6
#dense=16


In [ ]:
#max_seq = 4
#head=1
#dense_dim=4 70.92
#8 70.02
#16 69.87
#32 70.37
#64 70.08
#128 69.26
#256
#512
#1024

##max_seq = 2
#head=1
#dense_dim=4 69.94
#128 69.57

#head=4
#128 76.0
#64 74.36

#head=8
#128 74.57
#256 74.97

##max_seq = 10
#head=4
#dim=128 77.43
#64 76.71

#head=6
#64 76.87
#32 77.4
#1024 74.94
#16 77.64
#8 77.08

##max_seq =20
#head=6
#8 76.0
#32 76.71
#64 77.16
#128 77.4
#256